# Imports

In [4]:
import pandas as pd
import numpy as np

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score

%matplotlib inline

# Linear Discriminant Analysis

Let's return to the ${\tt Smarket}$ data.

In [2]:
df = pd.read_csv('Smarket.csv', usecols=range(1,10), index_col=0, parse_dates=True)
df.head()

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001-01-01,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001-01-01,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001-01-01,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001-01-01,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001-01-01,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


Now we will perform LDA on the ${\tt Smarket}$ data. In ${\tt Python}$, we can fit a LDA model using the ${\tt LDA()}$ function, which is part of the ${\tt lda}$ module of the ${\tt sklearn}$ library. As we did with logistic regression and KNN, we'll fit the model using only the observations before 2005, and then test the model on the data from 2005.

In [5]:
X_train = df[:'2004'][['Lag1','Lag2']]
y_train = df[:'2004']['Direction']

X_test = df['2005':][['Lag1','Lag2']]
y_test = df['2005':]['Direction']

lda = LDA()
model = lda.fit(X_train, y_train)

print(model.priors_)

[ 0.49198397  0.50801603]


The LDA output indicates prior probabilities of ${\hat{\pi}}_1 = 0.492$ and ${\hat{\pi}}_2 = 0.508$; in other words,
49.2% of the training observations correspond to days during which the
market went down.

In [26]:
print(model.means_)

[[ 0.04279022  0.03389409]
 [-0.03954635 -0.03132544]]


The above provides the group means; these are the average
of each predictor within each class, and are used by LDA as estimates
of $\mu_k$. These suggest that there is a tendency for the previous 2 days’
returns to be negative on days when the market increases, and a tendency
for the previous days’ returns to be positive on days when the market
declines. 

In [27]:
print(model.coef_)

[[-0.05544078 -0.0443452 ]]


The coefficients of linear discriminants output provides the linear
combination of ${\tt Lag1}$ and ${\tt Lag2}$ that are used to form the LDA decision rule.

If $−0.0554\times{\tt Lag1}−0.0443\times{\tt Lag2}$ is large, then the LDA classifier will
predict a market increase, and if it is small, then the LDA classifier will
predict a market decline. **Note**: these coefficients differ from those produced by ${\tt R}$.

The ${\tt predict()}$ function returns a list of LDA’s predictions about the movement of the market on the test data:

In [28]:
pred=model.predict(X_test)
print(np.unique(pred, return_counts=True))

(array(['Down', 'Up'], 
      dtype='<U4'), array([ 70, 182]))


The model assigned 70 observations to the "Down" class, and 182 observations to the "Up" class. Let's check out the confusion matrix to see how this model is doing. We'll want to compare the **predicted class** (which we can find in ${\tt pred}$) to the **true class** (found in ${\tt y\_test})$.

In [29]:
print(confusion_matrix(pred, y_test))
print(classification_report(y_test, pred, digits=3))

[[ 35  35]
 [ 76 106]]
             precision    recall  f1-score   support

       Down      0.500     0.315     0.387       111
         Up      0.582     0.752     0.656       141

avg / total      0.546     0.560     0.538       252



We can also get the predicted _probabilities_ using the ${\tt predict\_proba()}$ function:

In [30]:
pred_p = model.predict_proba(X_test)

Applying a 50% threshold to the posterior probabilities allows us to recreate
the predictions:

In [31]:
print(np.unique(pred_p[:,1]>0.5, return_counts=True))

(array([False,  True], dtype=bool), array([ 70, 182]))


Notice that the posterior probability output by the model corresponds to
the probability that the market will **increase**:

In [32]:
print(np.stack((pred_p[10:20,1], pred[10:20])).T)

[['0.5093037238790318' 'Up']
 ['0.4880011537380812' 'Down']
 ['0.510484773063352' 'Up']
 ['0.5293238777881214' 'Up']
 ['0.5255407143881711' 'Up']
 ['0.5200416608518921' 'Up']
 ['0.5064224705341396' 'Up']
 ['0.4969106228816935' 'Down']
 ['0.5021193878585957' 'Up']
 ['0.5113669134834818' 'Up']]


If we wanted to use a posterior probability threshold other than 50% in
order to make predictions, then we could easily do so. For instance, suppose
that we wish to predict a market decrease only if we are very certain that the
market will indeed decrease on that day—say, if the posterior probability
is at least 90%:

In [33]:
print(np.unique(pred_p[:,1]>0.9, return_counts=True))

(array([False], dtype=bool), array([252]))


No days in 2005 meet that threshold! In fact, the greatest posterior probability
of decrease in all of 2005 was 54.2%:

In [34]:
max(pred_p[:,1])

0.54221325545189769

# Quadratic Discriminant Analysis
We will now fit a QDA model to the ${\tt Smarket}$ data. QDA is implemented
in ${\tt sklearn}$ using the ${\tt QDA()}$ function, which is part of the ${\tt qda}$ module. The
syntax is identical to that of ${\tt LDA()}$.

In [36]:
qda = QDA()
model2 = qda.fit(X_train, y_train)
print(model2.priors_)
print(model2.means_)

[ 0.49198397  0.50801603]
[[ 0.04279022  0.03389409]
 [-0.03954635 -0.03132544]]


The output contains the group means. But it does not contain the coefficients
of the linear discriminants, because the QDA classifier involves a
_quadratic_, rather than a linear, function of the predictors. The ${\tt predict()}$
function works in exactly the same fashion as for LDA.

In [37]:
pred2=model2.predict(X_test)
print(np.unique(pred2, return_counts=True))
print(confusion_matrix(pred2, y_test))
print(classification_report(y_test, pred2, digits=3))

(array(['Down', 'Up'], dtype=object), array([ 50, 202]))
[[ 30  20]
 [ 81 121]]
             precision    recall  f1-score   support

       Down      0.600     0.270     0.373       111
         Up      0.599     0.858     0.706       141

avg / total      0.599     0.599     0.559       252



Interestingly, the QDA predictions are accurate almost 60% of the time,
even though the 2005 data was not used to fit the model. This level of accuracy
is quite impressive for stock market data, which is known to be quite
hard to model accurately. 

This suggests that the quadratic form assumed
by QDA may capture the true relationship more accurately than the linear
forms assumed by LDA and logistic regression. However, we recommend
evaluating this method’s performance on a larger test set before betting
that this approach will consistently beat the market!

# An Application to Carseats Data
Let's see how the ${\tt LDA/QDA}$ approach performs on the ${\tt Carseats}$ data set. 

Recall: this is a simulated data set containing sales of child car seats at 400 different stores.

In [38]:
df2 = pd.read_csv('Carseats.csv')
df2.head()

,Unnamed: 0,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,1,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,2,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,3,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,4,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,5,4.15,141,64,3,340,128,Bad,38,13,Yes,No


See if you can build a model that predicts ${\tt ShelveLoc}$, the shelf location (Bad, Good, or Medium) of the product at each store. Don't forget to hold out some of the data for testing!

In [21]:
# Your code here